In [4]:
import os
import urllib
import requests
import datetime as dt
import dateutil.parser as dparser
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)

# Get links from website
html = requests.get("https://bescom.karnataka.gov.in/new-page/Planned%20Outages%20-%20BESCOM%20Works/en").text
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
soup = BeautifulSoup(html)
links = [link.get('href').strip().replace(" ", "%20") for link in soup.find_all('a',href=True) if '.xlsx' in link.get('href') and 'Outage' in link.get('href')]
links

['https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/01-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/02-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/02-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/03-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/04-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/05-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/06-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/07-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/08-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/09-02-2022.xlsx',
 'https://bescom.karnataka.gov.in/storage/pdf-files/Planned%20Outages/10-02-2022.xlsx',
 'https://bescom.karnataka.gov.i

In [5]:
# Save files from links
file_path_list = []
for link in links:
    try:
        filename = link.split("/")[-1]
        urllib.request.urlretrieve(link, filename)
        file_path_list.append(filename)
        print(f"{filename} collected!")
    except Exception as e:
        print(f"{link} link broken?")
        print(f"{repr(e)}")

01-02-2022.xlsx collected!
02-02-2022.xlsx collected!
02-02-2022.xlsx collected!
03-02-2022.xlsx collected!
04-02-2022.xlsx collected!
05-02-2022.xlsx collected!
06-02-2022.xlsx collected!
07-02-2022.xlsx collected!
08-02-2022.xlsx collected!
09-02-2022.xlsx collected!
10-02-2022.xlsx collected!
11-02-2022.xlsx collected!
12-02-2022.xlsx collected!
13-02-2022.xlsx collected!
14-02-2022.xlsx collected!
15-02-2022.xlsx collected!
16-02-2022.xlsx collected!
17-02-2022.xlsx collected!
18-02-2022.xlsx collected!
19-02-2022.xlsx collected!
20-02-2022.xlsx collected!
21-02-2022.xlsx collected!
22-02-2022.xlsx collected!
23-02-2022.xlsx collected!
24-02-2022.xlsx collected!
25-02-2022.xlsx collected!
26-02-2022.xlsx collected!
27-02-2022.xlsx collected!
28-02-2022.xlsx collected!
KPTCL%20LC%20for%20the%20month%20of%20FEB%202022.xlsx collected!


In [6]:
# Fetch files
dfs = {}
for file in file_path_list:
    df = pd.read_excel(file)
    print(f"{file}  {len(df)} {len(df.keys())}")
    dfs[file] = df

01-02-2022.xlsx  60 31
02-02-2022.xlsx  173 22
02-02-2022.xlsx  173 22
03-02-2022.xlsx  203 22
04-02-2022.xlsx  208 22
05-02-2022.xlsx  205 22
06-02-2022.xlsx  134 22
07-02-2022.xlsx  254 22
08-02-2022.xlsx  192 22
09-02-2022.xlsx  209 22
10-02-2022.xlsx  204 22
11-02-2022.xlsx  192 22
12-02-2022.xlsx  103 22
13-02-2022.xlsx  95 22
14-02-2022.xlsx  204 22
15-02-2022.xlsx  176 22
16-02-2022.xlsx  133 22
17-02-2022.xlsx  156 22
18-02-2022.xlsx  147 22
19-02-2022.xlsx  131 22
20-02-2022.xlsx  63 22
21-02-2022.xlsx  136 22
22-02-2022.xlsx  122 22
23-02-2022.xlsx  122 22
24-02-2022.xlsx  119 22
25-02-2022.xlsx  124 22
26-02-2022.xlsx  43 22
27-02-2022.xlsx  43 22
28-02-2022.xlsx  103 22
KPTCL%20LC%20for%20the%20month%20of%20FEB%202022.xlsx  1 19


In [7]:
dfs_list = list(dfs.values())

# Mergability check
# assert all([len(dfs_list[0].columns.intersection(df.columns)) == dfs_list[0].shape[1] for df in dfs_list])

# Merge 
final_df = pd.concat(dfs_list, ignore_index=True).astype(str)
final_df.columns = final_df.columns.str.lower()
final_df.dropna(axis=0, how='all', inplace=True)

#Filter columns
important_columns = []
important_words = ['to', 'from', 'unit', 'date', 'feeder', 'nomen', 'area'
]
for col in final_df.columns:
    if any(important_word in col for important_word in important_words):
        important_columns.append(col)
        
final_df = final_df[important_columns]
specific_df = final_df[(final_df.values=='Marathalli').any(1)]
mask = np.column_stack([specific_df[col].str.contains("F10-TULASI-THEATER-ROAD", na=False) for col in specific_df])
specific_df = specific_df.loc[mask.any(axis=1)]
specific_df['date_date'] = None

for col in specific_df.columns:
    if 'date' in col:
        try:
            specific_df['date_date'] = specific_df[col].apply(lambda x: dparser.parse(x,fuzzy=True))
        except:
            continue
        

specific_df['date_date']

84     2022-02-02
668    2022-05-02
1661   2022-10-02
2261   2022-02-14
2924   2022-02-18
3516   2022-02-23
3874   2022-02-26
Name: date_date, dtype: datetime64[ns]

In [13]:
today = dt.datetime.now().date() - dt.timedelta(days=3)

In [18]:
str(today)

'2022-02-02'

In [20]:
str(today) in list(specific_df['date_date'].astype(str))

True